# Scratch notebook for experimenting with ridge trace algorithm

## Basic plan

0. We use a map of surface brightness, which we will call $B$
1. We start at some point on the emission arc that we want to trace. For instance, the local maximum brightness
2. Find the two perpendicular directions of the principal curvature: $\hat{n}_1$, $\hat{n}_2$
3. Take a step parallel to $\hat{n}_2$, which should be tangential to the arc
4. Then look along the direction $\hat{n}_1$ to find the point of maximum $B$ and set this as the next point on the arc
5. Return to step 2 and repeat
6. We need some criterion for saying when we should stop

In [8]:
import sys
from pathlib import Path
import numpy as np
from astropy.io import fits
from ridge_trace.curvature import ImageCurvature

In [5]:
sys.version_info

sys.version_info(major=3, minor=12, micro=1, releaselevel='final', serial=0)

## Load the image data from FITS files

In [6]:
datapath = Path.cwd().parent / "data"
fitsfiles = list(datapath.glob("*.fits"))
proplyd = "069-601"
hdu = fits.open(fitsfiles[1])["SCI"]

Calculate curvatures as in demo01 notebook

In [9]:
x1, x2 = 250, 420
y1, y2 = 250, 420
v0, d0 = 1.0, 20
ic = ImageCurvature(
    hdu.data, 
    smooth=2.5, 
    sharp=40, 
    max_cutoff=8.5, 
    bbox=[x1, y1, x2, y2], 
    scales=(v0, d0),
)

## Look at traces through the image from the peak


In [14]:
np.nanmin(ic.kappa1), 

-8.541056619175956

## Try to implement algorithm 

We are going to need a function to interpolate from the grid to an arbitrary point